# simple demo of ipyparallel
 - read more about ipyparallel architecture here
 - https://ipyparallel.readthedocs.io/en/latest/intro.html#architecture-overview

In [ ]:
import ipyparallel as ipp
import numpy as np

# to get process information
import os
import psutil

In [ ]:
! dir

In [ ]:
# ipcluster is part of ipyparallel
# help documentation is listed below
! ipcluster -h

In [ ]:
# start a new cluster (which is a controller and engines, the engines are what do the work)
# it is required to start a cluster before you can use ipyparallel
! ipcluster start -n 4

In [ ]:
# get some information on the process running jupyter notebook
this_process = os.getpid()
p = psutil.Process(this_process)
p

In [ ]:
# create an ipyparallel client
client = ipp.Client()

In [ ]:
# get client ids for the 4 engines
client.ids

In [ ]:
# run a simple lambda function on all 4 engines
client[:].apply_sync(lambda: "hello world")

In [ ]:
# run a simple lambda function on a specific engine
client[0].apply_sync(lambda: 2 ** 8)

In [ ]:
# create a direct view to execute on the engines using helper methods
# i used the 'map' function in this example
view = client[:]

In [ ]:
# print out process information on engines
for engine_pid in view.apply_sync(os.getpid):
    print(psutil.Process(engine_pid))

In [ ]:
# create an array of 1000 random numbers from 0 to 100,000
random_array = np.random.randint(100000, size=1000)

In [ ]:
# get number of engines/processes available
engine_count = len(client.ids)

In [ ]:
# split arrays into even segments based on number of engines
evenly_split_arrays = np.array_split(random_array, engine_count)

In [ ]:
evenly_split_arrays

In [ ]:
len(evenly_split_arrays)

In [ ]:
# setup blocking execution (blocking vs. non-blocking)
view.block = True

In [ ]:
# add up the segments
results = view.map(sum, evenly_split_arrays)

In [ ]:
results

In [ ]:
# total sum of all elements
sum(results)

In [ ]:
# verify results are the same as serial execution
sum(results) == sum(random_array)

In [ ]:
# shutdown engines and close client
! ipcluster stop

client.shutdown()
client.close()